In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import requests
from pprint import pprint
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import urlopen
from selenium.common.exceptions import NoSuchElementException    
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException   

In [6]:
data=pd.read_excel(r'C:\Users\aaparajit\Desktop\Projects\Marijuana Related Securities\Marijuana Test.xlsx',sheet_name=0,header=0)


In [7]:
data.head()

,U.S. Symbol,CUSIP,Security Name
0,TLRY,88688T100,TILRAY INCORPORATED COM CLASS 2
1,ACB,05156X108,AURORA CANNABIS INCORPORATED
2,CRON,22717L101,CRONOS GROUP INCORPORATED
3,NEPT,64079L105,NEPTUNE WELLNESS SOLUTIONS INCORPORATED
4,HEXO,428304109,HEXO CORPORATION


In [13]:
consolidated_data=pd.DataFrame()
for i in range(len(data)):
    symbol= data['U.S. Symbol'][i]
    cap = DesiredCapabilities().FIREFOX
    cap["marionette"] = True
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference('permissions.default.image', 2)
    firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
    firefox_profile.set_preference("browser.tabs.remote.autostart", False)
    firefox_profile.set_preference("browser.tabs.remote.autostart.1", False)
    firefox_profile.set_preference("browser.tabs.remote.autostart.2", False)
    binary = FirefoxBinary('C:/Users/aaparajit/AppData/Local/Mozilla Firefox/firefox.exe')
    driver= webdriver.Firefox(firefox_profile=firefox_profile,capabilities=cap,firefox_binary=binary,executable_path='C:/geckodriver-v0.24.0-win64/geckodriver.exe')
    search_query = symbol+" "+"Morningstar"+" "+"Ownership"
    driver.get("https://www.google.com/")
    search = driver.find_element_by_name('q')
    search.send_keys(search_query)
    search.send_keys(Keys.ENTER)
    time.sleep(5)
    try:
        matched_elements = driver.find_elements_by_xpath('//a[starts-with(@href, "https://www.morningstar.com")]')
        if matched_elements:
            matched_elements[0].click()
            driver.implicitly_wait(40)
            time.sleep(7)
            try:
                driver.find_element_by_xpath('/html/body/div[3]/main/div/div/section[4]/nav/div[2]/ul/li[8]/div/button').click() 
                time.sleep(10)
                lt = driver.find_elements_by_class_name('fixed-table')
                lt2=driver.find_elements_by_class_name('totalAssets')
                mf=[] 
                mf_asset=[]
                for a in range(len(lt)):
                        mf.append(lt[a].text.splitlines())     
                for a in range(len(lt2)):
                        mf_asset.append(lt2[a].text.splitlines())
                for j in range(len(mf[1][1:len(mf[1])-1])):
                        consolidated_data=consolidated_data.append({'Ticker':symbol,'MF':mf[1][1:len(mf[1])-1][j],'Share':mf_asset[2:len(mf_asset)-1][j]},ignore_index=True)
                driver.quit()
            except:
                pass
        if not matched_elements:
            driver.quit()
    except:
        pass

In [14]:
consolidated_data.to_csv(r'C:\Users\aaparajit\Desktop\MF_holdings.csv',encoding='utf-8', index=False)

In [15]:
consolidated_data.head()

,MF,Share,Ticker
0,ETFMG Alternative Harvest ETF,[6.15],TLRY
1,Horizons Marijuana Life Sciences ETF,[9.71],TLRY
2,Virtus Zevenbergen Innovative Gr Stk A,[3.59],TLRY
3,I.B.I TRK Solactive N.Amer Cannabis $Hdg,[7.47],TLRY
4,Federated Kaufmann Small Cap A,[0.09],TLRY
